# Langchain With Agents & Tools


In [ ]:
import sys
sys.path.insert(1, '../../')
import init_creds as creds
 
AZURE_OPENAI_API_KEY = creds.get_api_key()
AZURE_OPENAI_ENDPOINT = creds.get_endpoint()
# print(AZURE_OPENAI_API_KEY)
# print(AZURE_OPENAI_ENDPOINT)
 
if not AZURE_OPENAI_API_KEY:
    raise ValueError("No AZURE_OPENAI_API_KEY set for Azure OpenAI API")
if not AZURE_OPENAI_ENDPOINT:
    raise ValueError("No AZURE_OPENAI_ENDPOINT set for Azure OpenAI API")

In [ ]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o-mini"
os.environ["AZURE_OPENAI_API_VERSION"]="2024-07-01-preview"


from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

## Tool

In [ ]:
%pip install duckduckgo_search

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
response=search_tool.run("Who is Noushath Moidu");
print("response1------------")
print(response)
from langchain.tools import DuckDuckGoSearchResults
search_tool = DuckDuckGoSearchResults()
response=search_tool.run("Obama");
print("response2-------") 
print(response)

## Custom Tool

In [ ]:

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import BaseTool
from typing import Any
search_tool = DuckDuckGoSearchRun()
# single input tool

class Calculator(BaseTool):
    name:str = "Multiplier noush"
    description:str = "This tool multiplies the input number by 2, MAKE SURE TO PASS ONLY THE NUMBER AS INPUT"

    def _run(self, num : int) -> int:
        print('-----inside----Multiplier noush------')
        input = int(num)
        multiplier=2
        out= input * multiplier
    
        return out
    
    def _async_run(self) -> int:
        raise NotImplementedError("This tool does not support async run")

calc_tool=Calculator()
calc_tool.run({"num":100})

## Create Agents with tools

In [ ]:
llm = model
agent = initialize_agent([search_tool,Calculator()],
                          llm,
                            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
                            max_iterations=10,
                            handle_parsing_errors=True)


agent.run("Find How old is Narendra Modi, and give me the number multiplied by 2")

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import BaseTool

from typing import Any

# single input tool

class LengthOfWord(BaseTool):
    name:str = "Length of word"
    description:str = "Use this tool to calculate the length of a word."

    def _run(self, word : str) -> Any:
        print('-----inside---tool-------')
        return len(word) 
    
    def _async_run(self) -> Any:
        raise NotImplementedError("This tool does not support async run")

tools = [LengthOfWord()]


agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
    tools=tools,
    llm=model,
    max_iterations=3,
    early_stopping_method='generate',
    handle_parsing_errors=True,
)
agent("please tell me what is the length of word butter")
